In [2]:
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd

from src.sdg_11_3_1_src.sdg_11_3_1 import SDG11_3_1
from src.sdg_11_3_1_src.utils import InputFile

sdg = SDG11_3_1("", "C:/Users/Cusse/Scripts/geo_work/sdg-11-3-1")

In [9]:




merge_col = "lsoa11cd"
pop_cols = ["all_ages_2016", "all_ages_2019"]
land_cols = ["area_2016", "area_2019"]
years = 3
years_merge_suffixes = ("_16", "_19")
result_suffix = "16_19"

sdg.calculate_sdg(
    INPUT_FILES,
    merge_col,
    pop_cols,
    land_cols,
    years,
    years_merge_suffixes,
    result_suffix
)


TypeError: 'InputFile' object is not subscriptable

In [ ]:





pop_old = sdg.load_data(INPUT_FILES["pop_old"][0], INPUT_FILES["pop_old"][1], INPUT_FILES["pop_old"][2])
pop_new = sdg.load_data(INPUT_FILES["pop_new"][0], INPUT_FILES["pop_new"][1], INPUT_FILES["pop_new"][2])

pop_merge = sdg.merge_population_calc_pgr(pop_old, pop_new, merge_col, pop_cols, years_merge_suffixes, years)


land_area_old = sdg.load_data(INPUT_FILES["land_area_old"][0], INPUT_FILES["land_area_old"][1], INPUT_FILES["land_area_old"][2])
land_area_new = sdg.load_data(INPUT_FILES["land_area_new"][0], INPUT_FILES["land_area_new"][1], INPUT_FILES["land_area_new"][2])

filt_old_cond = (land_area_old['ctry'] == 'E') | (land_area_old['ctry'] == 'W') # filter to separate EW data
land_area_old_filtered= land_area_old.loc[filt_old_cond].copy()

filt_new_cond = (land_area_new['landcover_type']=='Manmade') # filter 'manmade' landcover for EW
land_area_new_filtered = land_area_new.loc[filt_new_cond]

land_area19_16ewmm = sdg.merge_land_consumption_calc_lcr(land_area_old_filtered, land_area_new_filtered, merge_col, land_cols, years_merge_suffixes, years)


lcrpop_merge = sdg.merge_lcr_pgr(pop_merge, land_area19_16ewmm, merge_col, [f"lcr_{result_suffix}", f"pgr_{result_suffix}"], ("_pgr", "_lcr"))

final_df = pd.concat([pop_merge, land_area19_16ewmm, lcrpop_merge], axis=1)
final_df.loc[:,~final_df.columns.duplicated()]

,all_ages_2016,all_ages_2019,pgr_16_19,area_2016,area_2019,lcr_16_19,pgr_lcr_16_19
lsoa11cd,,,,,,,
E01031349,1311,1344,0.008287,136747.07,137129.41240,0.000932,0.112469
E01031350,1182,1223,0.011366,82471.17,75879.30972,-0.026643,-2.344040
E01031351,1576,1576,0.000000,260859.31,347720.77350,0.110994,inf
E01031352,1544,1538,-0.001298,119586.74,102942.23400,-0.046395,35.746928
E01031370,1494,1545,0.011189,90438.90,97704.52581,0.026779,2.393357
...,...,...,...,...,...,...,...
E01013341,1336,1331,-0.001250,108388.53,109524.14660,0.003492,-2.794286
E01013342,1508,1490,-0.004003,319507.87,366331.17450,0.048849,-12.204062
E01013423,1294,1252,-0.010999,135120.39,177996.25620,0.105772,-9.616825


In [25]:
import geopandas as gpd

root_dir = "C:/Users/Cusse/Scripts/geo_work/sdg-11-3-1/sdg_11_3_1_data"

bua_geom_new = sdg.load_data(f"{root_dir}/ONS_BUA_2022_shp/BUA_2022_GB.shp", index="bua22cd")
bua_lu_new = sdg.load_data(f"{root_dir}/LSOA_(2021)_to_Built_Up_Area_to_Local_Authority_District_to_Region_(December_2022)_Lookup_in_England_and_Wales_v2.csv", index="bua22cd")

In [27]:
bua_geom_old = sdg.load_data(f"{root_dir}/Built_up_Area_Sub_Divisions_Dec_2011_Boundaries_2022_-1654302177174048508/BUASD_2011_EW_BGG.shp")
bua_lu_old = sdg.load_data(f"{root_dir}/Built_up_Areas_Dec_2011_Boundaries_V2_2022_3304006784241430068.csv")

In [40]:
bua_geom_old[""]

,buasd11cd,buasd11nm,buasd_id,bua_id,sd_type,globalid,geometry
0,E35001527,Lakeside Estate,5952,2817,Discrete,52cd33ec-6282-41e3-b748-7832d96c1fa2,"POLYGON ((504150.004 177099.995, 504150.004 17..."
1,E35001532,Brookfoot Quarry,5990,5422,Discrete,d87aecfd-16a7-42e5-8f74-67e67b19e55c,"POLYGON ((412900.000 423399.992, 412949.998 42..."
2,E35001546,"Clapham Hill, nr Whitstable",6102,4989,Discrete,848f42ce-6a4c-4196-84b9-fb509d1636a2,"POLYGON ((609750.001 163999.995, 609750.001 16..."
3,E35001547,"Works, nr Kingsnorth (Medway)",6105,3742,Discrete,a832331a-c663-4140-8aca-64b809b554d6,"POLYGON ((580949.986 172899.997, 580949.986 17..."
4,E35001550,Catsfield (North),6116,3879,Discrete,9b431df2-5000-43be-95a9-6c95e9a15f51,"POLYGON ((572649.996 114250.006, 572649.996 11..."
...,...,...,...,...,...,...,...
1821,E35000876,Waltham Cross BUASD,6925,5705,Joined,8efcd88e-c59d-4785-a95f-37bffc721acc,"POLYGON ((536600.002 199950.002, 536550.002 19..."
1822,E35001183,St Albans BUASD,7310,5705,Discrete,49f8740b-47ae-4212-80bd-988443cb668e,"MULTIPOLYGON (((516599.999 205150.000, 516599...."
1823,E35001396,Bramley BUASD,7546,5705,Discrete,2fbe3a6c-6bfb-41a6-8fa9-241bf298c9d9,"MULTIPOLYGON (((501100.001 145400.005, 501100...."
1824,E35000839,Hammersmith and Fulham BUASD,6882,5705,Joined,93feec3d-a0e6-4249-956d-154bd7e45474,"POLYGON ((521749.996 182800.005, 521799.996 18..."


In [3]:
root_dir ="C:/Users/Cusse/Scripts/geo_work/sdg-11-3-1"

def built_up_area_per_capita(bua, population):
    return bua / population


POP_INPUT_FILES = {
    "pop_t1": InputFile(f'{root_dir}/sdg_11_3_1_data/2019_2016_2013/population/SAPE_mid2013_ew.xls', {"sheet_name": "Mid-2013 Persons", "skiprows": 4, "index_col": "Area Codes", "thousands": ","}),
    "pop_t2": InputFile(f'{root_dir}/sdg_11_3_1_data/2019_2016_2013/population/popn-mid-2016-lsoa-ew.xls', {"sheet_name": "Mid-2016 Persons", "skiprows": 4, "index_col": "Area Codes", "thousands": ","}),
    "pop_t3": InputFile(f'{root_dir}/sdg_11_3_1_data/2019_2016_2013/population/popn-mid-2019-lsoa-ew.xlsx', {"sheet_name": "Mid-2019 Persons", "skiprows": 4, "engine": "openpyxl", "index_col": "LSOA Code", "thousands": ","}),
    "land_t1": InputFile(f'{root_dir}/sdg_11_3_1_data/2019_2016_2013/lsoa2016_2013_landcover_area_GB.csv', {"index_col": "LSOA11CD", "thousands": ","}),
    "land_t3": InputFile(f'{root_dir}/sdg_11_3_1_data/2019_2016_2013/lsoa2019_landcover_area_GB - Copy.csv', {"index_col": "LSOA11CD", "thousands": ","})
}

years = 3


pop_t1 = sdg.load_data(POP_INPUT_FILES["pop_t1"].file_path, kwargs=POP_INPUT_FILES["pop_t1"].kwargs).sort_index()
pop_t2 = sdg.load_data(POP_INPUT_FILES["pop_t2"].file_path, kwargs=POP_INPUT_FILES["pop_t2"].kwargs).sort_index()
pop_t3 = sdg.load_data(POP_INPUT_FILES["pop_t3"].file_path, kwargs=POP_INPUT_FILES["pop_t3"].kwargs).sort_index()

pop_t1 = pop_t1.rename(columns={"all ages": "all_ages_2013"})
pop_t2 = pop_t2.rename(columns={"all ages": "all_ages_2016"})
pop_t3 = pop_t3.rename(columns={"all ages": "all_ages_2019"})

pop_counts = [pop_t1["all_ages_2013"], pop_t2["all_ages_2016"], pop_t3["all_ages_2019"]]
comb_pop = pd.concat(pop_counts, axis=1).dropna()

t1_t2_pgr = sdg.population_growth_rate(comb_pop["all_ages_2013"], comb_pop["all_ages_2016"], years).rename("t1_t2_pgr")
t2_t3_pgr = sdg.population_growth_rate(comb_pop["all_ages_2016"], comb_pop["all_ages_2019"], years).rename("t2_t3_pgr")
pgrs = pd.concat([t1_t2_pgr,t2_t3_pgr], axis=1)

pop_calcs = pd.concat([comb_pop, pgrs], axis=1)


land_t1 = sdg.load_data(POP_INPUT_FILES["land_t1"].file_path, kwargs=POP_INPUT_FILES["land_t1"].kwargs)
land_t2 = sdg.load_data(POP_INPUT_FILES["land_t3"].file_path, kwargs=POP_INPUT_FILES["land_t3"].kwargs)

land_t2_mm = land_t2[land_t2["landcover_type"] == "Manmade"][["area_2019"]]

comb_land = pd.concat([land_t1[["area_2013", "area_2016"]], land_t2_mm], axis=1).dropna()

t1_t2_lcr = sdg.land_consumption_rate(comb_land["area_2013"], comb_land["area_2016"], years).rename("t1_t2_lcr")
t2_t3_lcr = sdg.land_consumption_rate(comb_land["area_2016"], comb_land["area_2019"], years).rename("t2_t3_lcr")
lcrs = pd.concat([t1_t2_lcr, t2_t3_lcr], axis=1)

land_calcs = pd.concat([comb_land, lcrs], axis=1)

comb_derived_values = pd.concat([pgrs, lcrs], axis=1)

lcr_pgr_ratio_t1_t2 = sdg.land_consumption_rate_population_growth_rate_ratio(comb_derived_values["t1_t2_lcr"], comb_derived_values["t1_t2_pgr"]).rename("lcr_pgr_ratio_t1_t2")
lcr_pgr_ratio_t2_t3 = sdg.land_consumption_rate_population_growth_rate_ratio(comb_derived_values["t2_t3_lcr"], comb_derived_values["t2_t3_pgr"]).rename("lcr_pgr_ratio_t2_t3")

final_values = pd.concat([lcr_pgr_ratio_t1_t2, lcr_pgr_ratio_t2_t3], axis=1)

full_report = pd.concat([pop_t3["lsoa name"], pop_calcs, land_calcs, final_values], axis=1)
full_report["bua_per_capita_t1"] = built_up_area_per_capita(full_report["area_2013"], full_report["all_ages_2013"])
full_report["bua_per_capita_t2"] = built_up_area_per_capita(full_report["area_2016"], full_report["all_ages_2016"])
full_report["bua_per_capita_t3"] = built_up_area_per_capita(full_report["area_2019"], full_report["all_ages_2019"])
full_report

c:\Users\Cusse\Anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,lsoa name,all_ages_2013,all_ages_2016,all_ages_2019,t1_t2_pgr,t2_t3_pgr,area_2013,area_2016,area_2019,t1_t2_lcr,t2_t3_lcr,lcr_pgr_ratio_t1_t2,lcr_pgr_ratio_t2_t3,bua_per_capita_t1,bua_per_capita_t2,bua_per_capita_t3
E01000001,City of London 001A,1216,1712,1636.0,0.114032,-0.015136,116142.47,116507.39,142088.12810,0.001047,0.073188,0.009185,-4.835338,95.511900,68.053382,86.850934
E01000002,City of London 001B,1240,1505,1558.0,0.064561,0.011537,206246.67,199430.64,142518.96470,-0.011016,-0.095124,-0.170630,-8.245315,166.327960,132.512053,91.475587
E01000003,City of London 001C,1109,1756,1786.0,0.153193,0.005647,54174.38,51603.17,34955.80132,-0.015821,-0.107535,-0.103272,-19.043911,48.849757,29.386771,19.572117
E01000005,City of London 001E,861,1722,1888.0,0.231049,0.030677,177250.86,169049.75,194423.47860,-0.015423,0.050032,-0.066751,1.630919,205.866272,98.170587,102.978537
E01000006,Barking and Dagenham 016A,1867,1857,2094.0,-0.001790,0.040038,80508.63,80508.63,71954.59214,0.000000,-0.035417,-0.000000,-0.884577,43.121923,43.354136,34.362269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
W01001954,Cardiff 006F,1550,1521,1485.0,-0.006296,-0.007984,184954.14,185019.82,223864.43780,0.000118,0.069983,-0.018802,-8.764927,119.325252,121.643537,150.750463
W01001955,Swansea 025F,1558,1829,1914.0,0.053455,0.015142,556841.43,562209.44,548012.68300,0.003213,-0.008417,0.060113,-0.555888,357.407850,307.386244,286.318016
W01001956,Swansea 023E,2689,2548,2276.0,-0.017954,-0.037630,215994.80,237298.56,240789.29730,0.032877,0.004903,-1.831224,-0.130308,80.325325,93.131303,105.794946
W01001957,Swansea 025G,1010,1120,1103.0,0.034459,-0.005098,145090.14,146623.59,155390.25300,0.003523,0.019930,0.102236,-3.909152,143.653604,130.913920,140.879649


In [4]:
full_report.to_csv("./full_report.csv")